In [167]:
from pymongo import MongoClient
import pandas as pd
import re
import geopandas as gpd

#Import MongoDB collection and create DataFrame

def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll

db, coll = connectCollection('companies','companies')

pipeline = [
    { "$unwind": "$offices"},    
 ]

results = list(coll.aggregate(pipeline))
df = pd.DataFrame(results)

#Drop deadpooled companies

df = df[df.deadpooled_year.isnull()]
df.reset_index(inplace=True, drop= True)

#Unwind Office column
df = pd.concat([df.drop(['offices'], axis=1), df['offices'].apply(pd.Series)], axis=1)

#Re-classifying categories into Tech/Other
tech = ["web", "games_video", "mobile", "social", "photo_video", "network_hosting", "software", "ecommerce", "hardware", "semiconductor", "analytics", "biotech", "cleantech", "nanotech"]
df["Tech/Other"] = df["category_code"].apply(lambda x: "Tech" if x in tech else "Other")

#Clean money raised column

def moneyRaise(value):
    dicc_coin = {'CAD': 0.76,'RUB': 0.016, 'EUR': 1.11, 'GBP': 1.29}
    values_money = {'K':1000, 'M':1000000, 'B': 100000000000}
    value_number = float(re.search('[+-]?([0-9]*[.])?[0-9]+', value)[0])
    if value.endswith('B'):
        exchange = value_number*(values_money['B'])
    elif value.endswith('K[k]'):
        exchange = value_number*(values_money['K'])
    elif value.endswith('M'):
        exchange = value_number*(values_money['M'])
    elif value.startswith("C"):
        exchange =  value_number*(dicc_coin['CAD'])
    elif value.startswith("$"):
        exchange =  value_number
    elif value.startswith("€"):
        exchange = value_number*(dicc_coin['EUR'])
    elif value.startswith("£"):
        exchange = value_number*(dicc_coin['GBP'])
    elif value.startswith("r"):
        exchange = value_number*(dicc_coin['RUB'])
    else:
        exchange = value_number
    return int(exchange)

df.total_money_raised = df.total_money_raised.apply(moneyRaise)

#Dropping rows with null coordinates

drop_rows = df[((df.latitude.isnull() == True) | (df.longitude.isnull() == True))].index
df.drop(drop_rows, inplace=True)



In [168]:
gdf_airports = gpd.read_file("./Input/ne_10m_airports/ne_10m_airports.shp")

#Create GeoJSON

def getLocation(gdf):
    long = gdf.x
    lat = gdf.y
    loc = {
        'type':'Point',
        'coordinates':[long, lat]
    }
    return loc

#Convert df in geoDataFrame

def df_to_gdf(dataframe):
    gdf = gpd.GeoDataFrame(dataframe, geometry=gpd.points_from_xy(dataframe.longitude, dataframe.latitude))
    gdf.crs = {'init' :'epsg:4326'}
    gdf.reset_index(drop=True, inplace=True)
    return gdf

gdf_master = df_to_gdf(df)

#Setting coordinates in GeoJson Format

gdf_master["geoJSON"] = gdf_master.geometry.apply(lambda x: getLocation(x))






In [173]:
gdf_master

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,...,address2,zip_code,city,state_code,country_code,latitude,longitude,Tech/Other,geometry,geoJSON
0,52cdef7c4bab8bd675297d90,Postini,postini,http://www.crunchbase.com/company/postini,http://postini.com,,,None,web,NaN,...,None,94070,San Carlos,CA,USA,37.506885,-122.247573,Tech,POINT (-122.24757 37.50688),"{'type': 'Point', 'coordinates': [-122.247573,..."
1,52cdef7c4bab8bd675297d91,Geni,geni,http://www.crunchbase.com/company/geni,http://www.geni.com,http://blog.geni.com,http://blog.geni.com/index.rdf,geni,web,18.0,...,,90069,West Hollywood,CA,USA,34.090368,-118.393064,Tech,POINT (-118.39306 34.09037),"{'type': 'Point', 'coordinates': [-118.393064,..."
2,52cdef7c4bab8bd675297d92,Flektor,flektor,http://www.crunchbase.com/company/flektor,http://www.flektor.com,http://www.flektor-blog.com,http://www.flektor-blog.com/video_editing_soft...,None,games_video,NaN,...,None,90232,Culver City,CA,USA,34.025958,-118.379768,Tech,POINT (-118.37977 34.02596),"{'type': 'Point', 'coordinates': [-118.379768,..."
3,52cdef7c4bab8bd675297d93,Fox Interactive Media,fox-interactive-media,http://www.crunchbase.com/company/fox-interact...,http://www.newscorp.com,,,twitterapi,web,0.0,...,,90210,Beverly Hills,CA,USA,34.076179,-118.394170,Tech,POINT (-118.39417 34.07618),"{'type': 'Point', 'coordinates': [-118.39417, ..."
4,52cdef7c4bab8bd675297d8d,Digg,digg,http://www.crunchbase.com/company/digg,http://www.digg.com,http://blog.digg.com/,http://blog.digg.com/?feed=rss2,digg,news,60.0,...,,94107,San Francisco,CA,USA,37.764726,-122.394523,Other,POINT (-122.39452 37.76473),"{'type': 'Point', 'coordinates': [-122.394523,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10160,52cdef7f4bab8bd67529c6f1,Willdan Group,willdan-group,http://www.crunchbase.com/company/willdan-group,http://www.willdan.com,,,,other,385.0,...,Suite 300,92806-5909,Anaheim,CA,USA,33.806525,-117.882314,Other,POINT (-117.88231 33.80653),"{'type': 'Point', 'coordinates': [-117.882314,..."
10161,52cdef7f4bab8bd67529c6f5,Oriact,oriact,http://www.crunchbase.com/company/oriact,http://www.oriact.com,,,,software,NaN,...,,6403,KÃ¼ssnacht,None,CHE,47.088219,8.437163,Tech,POINT (8.43716 47.08822),"{'type': 'Point', 'coordinates': [8.4371634, 4..."
10162,52cdef7f4bab8bd67529c6f7,AfterLogic,afterlogic,http://www.crunchbase.com/company/afterlogic,http://www.afterlogic.com,,,afterlogic,software,NaN,...,,07039,Livingston,NJ,USA,40.793024,-74.323554,Tech,POINT (-74.32355 40.79302),"{'type': 'Point', 'coordinates': [-74.3235539,..."
10163,52cdef7f4bab8bd67529c6f9,EnteGreat Solutions,entegreat-solutions,http://www.crunchbase.com/company/entegreat-so...,,,,,software,NaN,...,,,Birmingham,AL,USA,33.518885,-86.816068,Tech,POINT (-86.81607 33.51888),"{'type': 'Point', 'coordinates': [-86.816068, ..."


In [174]:
gdf_airports["geoJSON"] = gdf_airports.geometry.apply(lambda x: getLocation(x))
gdf_airports = gdf_airports.loc[:,["name", "geometry", "geoJSON"]]

In [175]:
gdf_airports["Tech/Other"] = "Airport"

In [176]:
gdf_airports

,name,geometry,geoJSON,Tech/Other
0,Sahnewal,POINT (75.95707 30.85036),"{'type': 'Point', 'coordinates': [75.957072240...",Airport
1,Solapur,POINT (75.93306 17.62542),"{'type': 'Point', 'coordinates': [75.933059771...",Airport
2,Birsa Munda,POINT (85.32360 23.31772),"{'type': 'Point', 'coordinates': [85.323597036...",Airport
3,Ahwaz,POINT (48.74711 31.34316),"{'type': 'Point', 'coordinates': [48.747106543...",Airport
4,Gwalior,POINT (78.21722 26.28549),"{'type': 'Point', 'coordinates': [78.217218654...",Airport
...,...,...,...,...
886,Madrid Barajas,POINT (-3.56903 40.46813),"{'type': 'Point', 'coordinates': [-3.569026654...",Airport
887,Luis Muñoz Marin,POINT (-66.00423 18.43808),"{'type': 'Point', 'coordinates': [-66.00422997...",Airport
888,Arlanda,POINT (17.93073 59.65112),"{'type': 'Point', 'coordinates': [17.930729901...",Airport
889,Soekarno-Hatta Int'l,POINT (106.65430 -6.12660),"{'type': 'Point', 'coordinates': [106.65429615...",Airport


In [179]:
df_master = pd.DataFrame(gdf_master)
df_airports = pd.DataFrame(gdf_airports)
# df_master.rename(columns={"description": "description2"}, inplace=True)
df_master.drop(df.columns[16], axis=1, inplace=True)
df_master.columns

Index(['_id', 'name', 'permalink', 'crunchbase_url', 'homepage_url',
       'blog_url', 'blog_feed_url', 'twitter_username', 'category_code',
       'number_of_employees', 'founded_year', 'deadpooled_year', 'tag_list',
       'alias_list', 'email_address', 'phone_number', 'created_at',
       'updated_at', 'overview', 'image', 'products', 'relationships',
       'competitions', 'providerships', 'total_money_raised', 'funding_rounds',
       'investments', 'acquisition', 'acquisitions', 'milestones',
       'video_embeds', 'screenshots', 'external_links', 'partners',
       'founded_month', 'founded_day', 'deadpooled_month', 'deadpooled_day',
       'deadpooled_url', 'ipo', 'address1', 'address2', 'zip_code', 'city',
       'state_code', 'country_code', 'latitude', 'longitude', 'Tech/Other',
       'geometry', 'geoJSON'],
      dtype='object')

In [180]:
df_output = df_master.append(df_airports, sort=False)

In [182]:
df_output.columns

Index(['_id', 'name', 'permalink', 'crunchbase_url', 'homepage_url',
       'blog_url', 'blog_feed_url', 'twitter_username', 'category_code',
       'number_of_employees', 'founded_year', 'deadpooled_year', 'tag_list',
       'alias_list', 'email_address', 'phone_number', 'created_at',
       'updated_at', 'overview', 'image', 'products', 'relationships',
       'competitions', 'providerships', 'total_money_raised', 'funding_rounds',
       'investments', 'acquisition', 'acquisitions', 'milestones',
       'video_embeds', 'screenshots', 'external_links', 'partners',
       'founded_month', 'founded_day', 'deadpooled_month', 'deadpooled_day',
       'deadpooled_url', 'ipo', 'address1', 'address2', 'zip_code', 'city',
       'state_code', 'country_code', 'latitude', 'longitude', 'Tech/Other',
       'geometry', 'geoJSON'],
      dtype='object')

In [183]:
df_output.reset_index(inplace=True, drop=True)
df_output.rename(columns={"_id": "ref"}, inplace=True)
display(df_output)

,ref,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,...,address2,zip_code,city,state_code,country_code,latitude,longitude,Tech/Other,geometry,geoJSON
0,52cdef7c4bab8bd675297d90,Postini,postini,http://www.crunchbase.com/company/postini,http://postini.com,,,None,web,NaN,...,None,94070,San Carlos,CA,USA,37.506885,-122.247573,Tech,POINT (-122.24757 37.50688),"{'type': 'Point', 'coordinates': [-122.247573,..."
1,52cdef7c4bab8bd675297d91,Geni,geni,http://www.crunchbase.com/company/geni,http://www.geni.com,http://blog.geni.com,http://blog.geni.com/index.rdf,geni,web,18.0,...,,90069,West Hollywood,CA,USA,34.090368,-118.393064,Tech,POINT (-118.39306 34.09037),"{'type': 'Point', 'coordinates': [-118.393064,..."
2,52cdef7c4bab8bd675297d92,Flektor,flektor,http://www.crunchbase.com/company/flektor,http://www.flektor.com,http://www.flektor-blog.com,http://www.flektor-blog.com/video_editing_soft...,None,games_video,NaN,...,None,90232,Culver City,CA,USA,34.025958,-118.379768,Tech,POINT (-118.37977 34.02596),"{'type': 'Point', 'coordinates': [-118.379768,..."
3,52cdef7c4bab8bd675297d93,Fox Interactive Media,fox-interactive-media,http://www.crunchbase.com/company/fox-interact...,http://www.newscorp.com,,,twitterapi,web,0.0,...,,90210,Beverly Hills,CA,USA,34.076179,-118.394170,Tech,POINT (-118.39417 34.07618),"{'type': 'Point', 'coordinates': [-118.39417, ..."
4,52cdef7c4bab8bd675297d8d,Digg,digg,http://www.crunchbase.com/company/digg,http://www.digg.com,http://blog.digg.com/,http://blog.digg.com/?feed=rss2,digg,news,60.0,...,,94107,San Francisco,CA,USA,37.764726,-122.394523,Other,POINT (-122.39452 37.76473),"{'type': 'Point', 'coordinates': [-122.394523,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11051,NaN,Madrid Barajas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Airport,POINT (-3.56903 40.46813),"{'type': 'Point', 'coordinates': [-3.569026654..."
11052,NaN,Luis Muñoz Marin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Airport,POINT (-66.00423 18.43808),"{'type': 'Point', 'coordinates': [-66.00422997..."
11053,NaN,Arlanda,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Airport,POINT (17.93073 59.65112),"{'type': 'Point', 'coordinates': [17.930729901..."
11054,NaN,Soekarno-Hatta Int'l,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Airport,POINT (106.65430 -6.12660),"{'type': 'Point', 'coordinates': [106.65429615..."


In [164]:
df_output.drop(df_output.columns[-2], axis=1, inplace=True)
df_output.drop(df_output.columns[0], axis=1, inplace=True)
print(df_output.dtypes)
df_output.to_json("updated_collection.json", orient='records')

name                    object
permalink               object
crunchbase_url          object
homepage_url            object
blog_url                object
blog_feed_url           object
twitter_username        object
category_code           object
number_of_employees    float64
founded_year           float64
deadpooled_year        float64
tag_list                object
alias_list              object
email_address           object
phone_number            object
created_at              object
updated_at              object
overview                object
image                   object
products                object
relationships           object
competitions            object
providerships           object
total_money_raised     float64
funding_rounds          object
investments             object
acquisition             object
acquisitions            object
milestones              object
video_embeds            object
screenshots             object
external_links          object
partners

In [165]:
df_output

,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,founded_year,...,ipo,address1,address2,zip_code,city,state_code,country_code,latitude,longitude,geoJSON
0,Postini,postini,http://www.crunchbase.com/company/postini,http://postini.com,,,None,web,NaN,1999.0,...,None,"959 Skyway Road, Suite 200",None,94070,San Carlos,CA,USA,37.506885,-122.247573,"{'type': 'Point', 'coordinates': [-122.247573,..."
1,Geni,geni,http://www.crunchbase.com/company/geni,http://www.geni.com,http://blog.geni.com,http://blog.geni.com/index.rdf,geni,web,18.0,2006.0,...,None,9229 W. Sunset Blvd.,,90069,West Hollywood,CA,USA,34.090368,-118.393064,"{'type': 'Point', 'coordinates': [-118.393064,..."
2,Flektor,flektor,http://www.crunchbase.com/company/flektor,http://www.flektor.com,http://www.flektor-blog.com,http://www.flektor-blog.com/video_editing_soft...,None,games_video,NaN,NaN,...,None,"8536 National Blvd, Suite A",None,90232,Culver City,CA,USA,34.025958,-118.379768,"{'type': 'Point', 'coordinates': [-118.379768,..."
3,Fox Interactive Media,fox-interactive-media,http://www.crunchbase.com/company/fox-interact...,http://www.newscorp.com,,,twitterapi,web,0.0,1979.0,...,None,407 N Maple Dr,,90210,Beverly Hills,CA,USA,34.076179,-118.394170,"{'type': 'Point', 'coordinates': [-118.39417, ..."
4,Digg,digg,http://www.crunchbase.com/company/digg,http://www.digg.com,http://blog.digg.com/,http://blog.digg.com/?feed=rss2,digg,news,60.0,2004.0,...,None,135 Mississippi St,,94107,San Francisco,CA,USA,37.764726,-122.394523,"{'type': 'Point', 'coordinates': [-122.394523,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11051,Madrid Barajas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'type': 'Point', 'coordinates': [-3.569026654..."
11052,Luis Muñoz Marin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'type': 'Point', 'coordinates': [-66.00422997..."
11053,Arlanda,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'type': 'Point', 'coordinates': [17.930729901..."
11054,Soekarno-Hatta Int'l,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'type': 'Point', 'coordinates': [106.65429615..."


In [ ]:
df_output.columns

In [166]:
df_output.columns

Index(['name', 'permalink', 'crunchbase_url', 'homepage_url', 'blog_url',
       'blog_feed_url', 'twitter_username', 'category_code',
       'number_of_employees', 'founded_year', 'deadpooled_year', 'tag_list',
       'alias_list', 'email_address', 'phone_number', 'created_at',
       'updated_at', 'overview', 'image', 'products', 'relationships',
       'competitions', 'providerships', 'total_money_raised', 'funding_rounds',
       'investments', 'acquisition', 'acquisitions', 'milestones',
       'video_embeds', 'screenshots', 'external_links', 'partners',
       'founded_month', 'founded_day', 'deadpooled_month', 'deadpooled_day',
       'deadpooled_url', 'ipo', 'address1', 'address2', 'zip_code', 'city',
       'state_code', 'country_code', 'latitude', 'longitude', 'geoJSON'],
      dtype='object')

In [161]:
df_master.geometry

0        POINT (-122.24757 37.50688)
1        POINT (-118.39306 34.09037)
2        POINT (-118.37977 34.02596)
3        POINT (-118.39417 34.07618)
4        POINT (-122.39452 37.76473)
                    ...             
10160    POINT (-117.88231 33.80653)
10161       POINT (8.43716 47.08822)
10162     POINT (-74.32355 40.79302)
10163     POINT (-86.81607 33.51888)
10164     POINT (-86.76130 33.40779)
Name: geometry, Length: 10165, dtype: geometry

In [153]:
df_output

ref                      object
name                     object
permalink                object
crunchbase_url           object
homepage_url             object
blog_url                 object
blog_feed_url            object
twitter_username         object
category_code            object
number_of_employees     float64
founded_year            float64
deadpooled_year         float64
tag_list                 object
alias_list               object
email_address            object
phone_number             object
created_at               object
updated_at               object
overview                 object
image                    object
products                 object
relationships            object
competitions             object
providerships            object
total_money_raised      float64
funding_rounds           object
investments              object
acquisition              object
acquisitions             object
milestones               object
video_embeds             object
screensh